<a href="https://colab.research.google.com/github/28humaid/book_recomm_system/blob/main/colaborativeFiltering_bookRecomm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
books = pd.read_csv('/content/drive/MyDrive/book_recommender/input/Books.csv')
users = pd.read_csv('/content/drive/MyDrive/book_recommender/input/Users.csv')
ratings = pd.read_csv('/content/drive/MyDrive/book_recommender/input/Ratings.csv')

<ipython-input-2-c39b0eb75fb1>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('/content/drive/MyDrive/book_recommender/input/Books.csv')


In [3]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [5]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


## check the shape of data

In [6]:
print(books.shape)
print(users.shape)
print(ratings.shape)

(271360, 8)
(278858, 3)
(1149780, 3)


## check null values

In [7]:
print(books.isnull().sum())
print('-----------------------')
print(users.isnull().sum())
print('-----------------------')
print(ratings.isnull().sum())

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64
-----------------------
User-ID          0
Location         0
Age         110762
dtype: int64
-----------------------
User-ID        0
ISBN           0
Book-Rating    0
dtype: int64


## check duplicate values

In [8]:
print(books.duplicated().sum())
print(users.duplicated().sum())
print(ratings.duplicated().sum())

0
0
0


# Collaborative filtering based recommender system


In [9]:
# consider books jinpr atleast 50 users have rated
# consider users jinhone atleast 200 books ko rate kiya hai

In [10]:
# pehle users k basis pr filter kr lo
relevantUsers=ratings.groupby('User-ID').count()['ISBN']>200
# user-ID as index mil jayegi
relevantUsers=relevantUsers[relevantUsers].index
relevantUsers

Index([   254,   2276,   2766,   2977,   3363,   3757,   4017,   4385,   6242,
         6251,
       ...
       274004, 274061, 274301, 274308, 274808, 275970, 277427, 277478, 277639,
       278418],
      dtype='int64', name='User-ID', length=899)

In [11]:
filtered_ratings=ratings[ratings['User-ID'].isin(relevantUsers)]
filtered_ratings

,User-ID,ISBN,Book-Rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0
...,...,...,...
1147612,275970,3829021860,0
1147613,275970,4770019572,0
1147614,275970,896086097,0
1147615,275970,9626340762,8


In [12]:
# ab isme se wo books le le
relevantBooks=filtered_ratings.groupby('ISBN').count()['Book-Rating'] > 50
#boolean series milegi
relevantBooks=relevantBooks[relevantBooks].index
#ab usko as index treat krne k liye nikal le
relevantBooks

Index(['002542730X', '006016848X', '0060391626', '0060392452', '0060502258',
       '0060915544', '0060921145', '0060928336', '0060930535', '0060934417',
       ...
       '1400031354', '1400031362', '1400034779', '155874262X', '1558743669',
       '1558744150', '1558745157', '1559029838', '1573225789', '1573229326'],
      dtype='object', name='ISBN', length=504)

In [13]:
final_ratings=filtered_ratings[filtered_ratings['ISBN'].isin(relevantBooks)]
final_ratings

,User-ID,ISBN,Book-Rating
1456,277427,002542730X,10
1469,277427,0060930535,0
1471,277427,0060934417,0
1474,277427,0061009059,9
1484,277427,0140067477,0
...,...,...,...
1147208,275970,0743412028,0
1147304,275970,0804111359,0
1147436,275970,140003065X,0
1147440,275970,1400031354,0


In [14]:
#ab isse pivot table bna lo
pivot_table=final_ratings.pivot(index='ISBN',columns='User-ID',values='Book-Rating')
pivot_table

User-ID,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
ISBN,,,,,,,,,,,,,,,,,,,,,
002542730X,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN
006016848X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
0060391626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0060392452,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,10.0,NaN,...,9.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
0060502258,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1558744150,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1558745157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1559029838,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [15]:
pivot_table.fillna(0,inplace=True)
pivot_table

User-ID,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
ISBN,,,,,,,,,,,,,,,,,,,,,
002542730X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0
006016848X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060391626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060392452,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,...,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060502258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1558744150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1558745157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1559029838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# genuine authors aur famous books, dono mil gye
# ab ek row ko (yani ki ek book) 883 dimensional point consider kro
# ksi bhi book liye, cosine similarity lga kr...panch "883 dimensional point" (books) nikal lo

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
similarity_scores=cosine_similarity(pivot_table)

In [19]:
similarity_scores

array([[1.        , 0.04948941, 0.05746684, ..., 0.05044148, 0.04263472,
        0.13020411],
       [0.04948941, 1.        , 0.10048799, ..., 0.08820327, 0.07455217,
        0.07244309],
       [0.05746684, 0.10048799, 1.        , ..., 0.10242116, 0.15582523,
        0.08412052],
       ...,
       [0.05044148, 0.08820327, 0.10242116, ..., 1.        , 0.1367755 ,
        0.07383673],
       [0.04263472, 0.07455217, 0.15582523, ..., 0.1367755 , 1.        ,
        0.09985459],
       [0.13020411, 0.07244309, 0.08412052, ..., 0.07383673, 0.09985459,
        1.        ]])

In [20]:
similarity_scores.shape

(504, 504)

In [27]:
# function to give 10 books based on this cosine similarity
listOfISBNs=[]

def recommend(book_name):
  index=np.where(pivot_table.index==book_name)[0][0]
  similar_items=sorted(list(enumerate(similarity_scores[0])),key=lambda x:x[1],reverse=True)[1:11]

  for i in similar_items:
    listOfISBNs.append(pivot_table.index[i[0]])

In [29]:
recommend('0060391626')
listOfISBNs

['0446608890',
 '0804108749',
 '080411109X',
 '0345422317',
 '0385492081',
 '0060958022',
 '0451184963',
 '067102423X',
 '0449221490',
 '0553574574']